In [18]:
import pandas as pd
import matplotlib as mpl
import geopandas as gpd
import descartes
import geojson
import json
import altair as alt
import altair_latimes as lat
alt.renderers.enable('notebook')
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')

ThemeRegistry.enable('latimes')

In [19]:
pd.options.display.float_format = '{:,}'.format

In [20]:
claims = pd.read_csv('claims.csv')
claims.columns = claims.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

In [21]:
claims['entry_date'] = pd.to_datetime(claims['entry_date'])
claims['year'] = claims['entry_date'].dt.year
claims['month'] = claims['entry_date'].dt.month

In [22]:
claims.head(2)

,file_number,entry_date,claimant_name,case_number,case_name,department_name,coverage_detail_desc,amount_paid,year,month
0,07-1052771*001,2007-05-10,"HAGER, DARREN",BC370326,"HAGER, DARREN VS COUNTY OF LOS ANGELES; ET AL",SHERIFF,EMPLOYMENT PRACTICES,3098662,2007,5
1,08-1062459*001,2008-07-08,"CASILLAS, NANCY",NaN,NaN,SHERIFF,POLICE LIABILITY/LAW ENFORCEMENT,2500,2008,7


In [23]:
claims['amount_paid'] = claims['amount_paid'].astype(float)

In [24]:
claims.groupby('department_name').agg({'amount_paid': 'sum'}).sort_values('amount_paid', ascending=False).reset_index()

,department_name,amount_paid
0,SHERIFF,"222,686,897.0"
1,HEALTH SERVICES (DHS),"77,911,314.0"
2,PUBLIC WORKS,"25,372,841.0"
3,CHILDREN AND FAMILY SERVICES (DCFS),"14,989,292.0"
4,FIRE (FD),"14,176,509.0"
5,PROBATION,"7,959,982.0"
6,PARKS AND RECREATION,"7,670,828.0"
7,CHIEF EXECUTIVE OFFICE (CEO),"6,373,416.0"
8,AUDITOR-CONTROLLER,"5,996,942.0"
9,MENTAL HEALTH,"5,600,692.0"


In [25]:
claims.set_index('department_name', inplace=True)

In [26]:
sheriff = claims.loc['SHERIFF']
dcfs = claims.loc['CHILDREN AND FAMILY SERVICES (DCFS)']
probation = claims.loc['PROBATION']
public_works = claims.loc['PUBLIC WORKS'] 
fire = claims.loc['FIRE (FD)']
dhs = claims.loc['HEALTH SERVICES (DHS)']

In [27]:
dhs.groupby(['year']).agg({'amount_paid': 'sum'}).sort_values(['year'], ascending=False).round().reset_index()

,year,amount_paid
0,2018,"690,646.0"
1,2017,"885,497.0"
2,2016,"6,270,032.0"
3,2015,"6,896,750.0"
4,2014,"7,162,415.0"
5,2013,"10,280,008.0"
6,2012,"17,936,281.0"
7,2011,"6,629,087.0"
8,2010,"7,557,879.0"
9,2009,"12,848,511.0"


In [28]:
fire.groupby('coverage_detail_desc').agg({'amount_paid': 'sum'}).sort_values('amount_paid', ascending=False).round().reset_index()

,coverage_detail_desc,amount_paid
0,MEDICAL MALPRACTICE,"5,563,718.0"
1,AUTO LIABILITY,"5,162,694.0"
2,EMPLOYMENT PRACTICES,"3,096,790.0"
3,GENERAL LIABILITY,"351,802.0"
4,ENVIRONMENTAL IMPAIRMENT LIABILITY,"1,355.0"
5,POLICE LIABILITY/LAW ENFORCEMENT,150.0


In [29]:
claims.groupby('coverage_detail_desc').agg({'amount_paid': 'sum'}).sort_values('amount_paid', ascending=False).round().reset_index()

,coverage_detail_desc,amount_paid
0,POLICE LIABILITY/LAW ENFORCEMENT,"171,622,919.0"
1,MEDICAL MALPRACTICE,"64,446,426.0"
2,GENERAL LIABILITY,"62,527,828.0"
3,AUTO LIABILITY,"54,592,524.0"
4,EMPLOYMENT PRACTICES,"38,775,349.0"
5,PROFESSIONAL LIABILITY - NOT OTHERWISE CLASSIFIED,"16,805,522.0"
6,ENVIRONMENTAL IMPAIRMENT LIABILITY,"73,455.0"
7,PUBLIC OFFICIALS ERRORS AND OMISSIONS,"41,061.0"
8,HOSPITAL PROFESSIONAL LIABILITY,"21,482.0"
9,PERMITTEE DRIVER PERSONAL VEHICLE DAMAGE,"19,762.0"
